In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [5]:
cc_data = pd.read_excel('dataset_ClimateChange/climate_change_dataset.xlsx')
cc_data.head()

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,...,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.57481,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.967875,..,..,..,...,..,..,..,..,..,..,..,..,..,..


Convert the excel data to long format and clean raw value

In [8]:
year_cols = [c for c in cc_data.columns if str(c).isdigit()]
long_cc_data = cc_data.melt(
    id_vars=["Country code", "Country name", "Series code", "Series name"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)
long_cc_data["Year"] = pd.to_numeric(long_cc_data["Year"], errors="coerce")
long_cc_data["Value"] = pd.to_numeric(long_cc_data["Value"].replace("..", np.nan), errors="coerce")
print("Long shape:", long_cc_data.shape)
long_cc_data.head(200)

Long shape: (297264, 6)


,Country code,Country name,Series code,Series name,Year,Value
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,29.574810
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,0.000000
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,0.000000
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,0.208235
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,4.967875
...,...,...,...,...,...,...
195,SVN,Slovenia,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,0.204426
196,SWE,Sweden,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,1.528626
197,SWZ,Swaziland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,0.000000
198,SXM,Sint Maarten (Dutch part),AG.LND.EL5M.ZS,Land area below 5m (% of land area),1990,NaN


In [17]:
codes_needed = ["NY.GDP.MKTP.CD", "SP.POP.TOTL", "EG.USE.PCAP.KG.OE", "EN.ATM.CO2E.KT"]
filtered = long_cc_data_cleaned[long_cc_data_cleaned["Series code"].isin(codes_needed)].copy()
data = filtered.pivot_table(
    index=["Country code", "Country name", "Year"],
    columns="Series code",
    values="Value"
).reset_index()
data.columns.name = None
print("Wide shape:", data.shape)
data.head()

Wide shape: (4877, 7)


,Country code,Country name,Year,EG.USE.PCAP.KG.OE,EN.ATM.CO2E.KT,NY.GDP.MKTP.CD,SP.POP.TOTL
0,ABW,Aruba,1990,NaN,1840.834,NaN,62147.0
1,ABW,Aruba,1991,NaN,1928.842,8.720670e+08,64642.0
2,ABW,Aruba,1992,NaN,1723.490,9.586592e+08,68286.0
3,ABW,Aruba,1993,NaN,1771.161,1.083240e+09,72573.0
4,ABW,Aruba,1994,NaN,1763.827,1.245810e+09,76762.0


In [18]:

data = data.rename(columns={
    "NY.GDP.MKTP.CD": "GDP",
    "SP.POP.TOTL": "Population",
    "EG.USE.PCAP.KG.OE": "EnergyUse_perCapita",
    "EN.ATM.CO2E.KT": "CO2_Emissions"
})
print(data.shape)
data.head()

(4877, 7)


,Country code,Country name,Year,EnergyUse_perCapita,CO2_Emissions,GDP,Population
0,ABW,Aruba,1990,NaN,1840.834,NaN,62147.0
1,ABW,Aruba,1991,NaN,1928.842,8.720670e+08,64642.0
2,ABW,Aruba,1992,NaN,1723.490,9.586592e+08,68286.0
3,ABW,Aruba,1993,NaN,1771.161,1.083240e+09,72573.0
4,ABW,Aruba,1994,NaN,1763.827,1.245810e+09,76762.0


In [19]:
data_cleaned = data.dropna(how='any')
data_cleaned = data_cleaned.reset_index(drop=True)
print("Before cleaning:", len(data), "rows")
print("After cleaning:", len(data_cleaned), "rows")
data_cleaned.head(200)

Before cleaning: 4877 rows
After cleaning: 2847 rows


,Country code,Country name,Year,EnergyUse_perCapita,CO2_Emissions,GDP,Population
0,AGO,Angola,1990,569.242999,4429.736,1.026019e+10,10335052.0
1,AGO,Angola,1991,565.451027,4367.397,1.219375e+10,10653515.0
2,AGO,Angola,1992,550.548264,4418.735,5.779394e+09,11002910.0
3,AGO,Angola,1993,551.688878,5801.194,5.285600e+09,11371750.0
4,AGO,Angola,1994,541.553067,3890.687,4.059577e+09,11742960.0
...,...,...,...,...,...,...,...
195,BGD,Bangladesh,1994,126.988772,18969.391,3.376866e+10,115059015.0
196,BGD,Bangladesh,1995,135.307621,22816.074,3.793975e+10,117486952.0
197,BGD,Bangladesh,1996,133.336186,24029.851,4.066602e+10,119928682.0
198,BGD,Bangladesh,1997,136.512119,25063.945,4.231880e+10,122370403.0


In [20]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2847 entries, 0 to 2846
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country code         2847 non-null   object 
 1   Country name         2847 non-null   object 
 2   Year                 2847 non-null   int64  
 3   EnergyUse_perCapita  2847 non-null   float64
 4   CO2_Emissions        2847 non-null   float64
 5   GDP                  2847 non-null   float64
 6   Population           2847 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 155.8+ KB
